### Prerequisites
Our goal for this tutorial is to learn how to look at data stored in a ROOT file. We'll access the ROOT file using `uproot`. We can use `numpy` for the analysis, and we can make plots with `matplotlib`. You can install all of these easily using `pip`.

The data used in this tutorial is from the LHCb opendata portal:

https://opendata.cern.ch/record/4900#

Here you can find additional files as well as another sample analysis.

In [ ]:
import uproot
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

### Exploring a ROOT file.
We're going to study $B^{\pm}$ mesons decaying to $K^{\pm}K^{\mp}K^{\pm}$ (called "kaons"). The $B^{\pm}$ will travel a few milimeters on average and decay before reaching any detector material. The three kaons will travel through the entire detector, producing series of signals that lets us reconstruct their trajectories. They'll be deflected by the magnetic field, allowing us to measure their momenta. We can use their reconstructed momenta to reconstruct the parent $B^{\pm}$ mesons.

We have a ROOT file containing $B^{\pm}\to h^{\pm}h^{\mp}h^{\pm}$ candidates from real data recorded by the LHCb detector. Here $h$ stands for any long-lived charged hadron, so charged pions, kaons, or protons. We'll use this data to do a simple analysis. First, we have to open the file. We can do this with `uproot`. Then we can use the `items()` method to list the file's contents.

In [ ]:
f = uproot.open('/share/lazy/tboettch/tutorial/B2HHH_MagnetDown.root')
f.items()

This file contains one object called `'DecayTree;1'`. This object is called a `TTree`. `TTree`s are how we typically store data used for particle physics analyses. Each entry in this `TTree` is a $B^{\pm}$ candidate, and the `TTree` has different `branches` that hold quantities that describe the candidate. We can think of this like a spreadsheet: each column is a different `branch`, and each row is an entry. We can load this `TTree` using `uproot` and see what branches it contains.

In [ ]:
t = f['DecayTree']
t.items()

You can see that this `TTree` contains a couple of branches with information on the $B^{\pm}$ itself, and branches describing the final-state hadrons. Each final-state hadron has branches giving its momentum (`PX`, `PY`, `PZ`) and information about how the particle has been identified (`ProbK`, `ProbPi`, `isMuon`). Each hadron also has a `Charge` and `IPChi2`. The charge is self-explanatory, but the `IPChi2` is more complicated. The `IPChi2` quantifies how well a charged particle points back to the $pp$ collision that produced it. Because $B$ mesons fly a significant distance before decaying, their decay products will tend to have relatively large `IPChi2`s.

### Working with the data
To work with the data in the `TTree`, we have to load it into arrays.

In [ ]:
arrs = t.arrays()
print(arrs)

By default, `uproot` loads data into `awkward` arrays. The `awkward` package has a lot of nice features for handling particle physics data, but we won't need most of them. In fact, you could load the data using some other library like `numpy` or `pandas` using:

```
arrs = t.arrays(library='np')
arrs = t.arrays(library='pd')
```

You may have noticed that the `TTree` doesn't contain any information about the `B` mesons' momenta or masses. This is because we have to calculate it ourselves using the momenta of the final-state particles. To do this, we need each particle's four-momentum. The four-momentum is a four-component vector containing $(p_x, p_y, p_z, E)$. Each component of the four-momentum is conserved in every particle interaction or decay. Four-momenta will obey the relativistic relation
$$
E^2=p^2+m^2
$$
where $p^2=p_x^2+p_y^2+p_z^2$ and $m$ is the particle's mass. For charged particles, we make our best guess of the particle ID, and use that to assign a mass. For now, we'll assume the final-state hadrons are kaons, so they'll have the kaon mass of about $497.6~{\rm MeV}$. 

Let's start by defining four-momenta for one of the final-state particles. 

#### Note: Vectorization
When we perform operations on `awkward` or `numpy` arrays like the ones we just loaded, those operations will be vectorized. This means that when we perform an operation like `array3 = array1 + array2`, the computer will perform the same operation in parallel for different elements of the array. Vectorized operations like this will be much faster than using a loop, such as:
```
for i in range(len(array1)):
    array3[i] = array1[i] + array2[i]
```
Python has a reputation for being slow, but if you use vectorization your code will generally be competitive with high-performance languages like C++. In general, if you find yourself writing a loop to perform a calculation with a large data array, there's probably a faster way to do it!

In [ ]:
m_k = 497.611
h1_px = arrs['H1_PX']
h1_py = arrs['H1_PY']
h1_pz = arrs['H1_PZ']
h1_p = np.sqrt(h1_px**2 + h1_py**2 + h1_pz**2)
h1_e = np.sqrt(h1_p**2 + m_k**2)
h1_p4 = np.array([h1_px, h1_py, h1_pz, h1_e])


Here we've created a 4xN array, where N is the number of decay candidates. If you're feeling adventurous, you can try creating a four-momentum class that takes $p_x$, $p_y$, $p_z$, and $E$ and calculates useful quantities like $m$ and $p$ for you.

Next we can define four-momenta for the other two final-state particles.

In [ ]:
h2_px = arrs['H2_PX']
h2_py = arrs['H2_PY']
h2_pz = arrs['H2_PZ']
h2_p = np.sqrt(h2_px**2 + h2_py**2 + h2_pz**2)
h2_e = np.sqrt(h2_p**2 + m_k**2)
h2_p4 = np.array([h2_px, h2_py, h2_pz, h2_e])

h3_px = arrs['H3_PX']
h3_py = arrs['H3_PY']
h3_pz = arrs['H3_PZ']
h3_p = np.sqrt(h3_px**2 + h3_py**2 + h3_pz**2)
h3_e = np.sqrt(h3_p**2 + m_k**2)
h3_p4 = np.array([h3_px, h3_py, h3_pz, h3_e])

Now that we have four-momenta for each of the final-state particles, we can calculate four-momenta of the $B$ meson. Then we can use the $B$ mesons four-momentum to calculate its mass. For convenience later, we'll add the $B$ mass back to `arrs` as a new variable.

In [ ]:
b_p4 = h1_p4 + h2_p4 + h3_p4
b_m = np.sqrt(b_p4[3]**2 - b_p4[0]**2 - b_p4[1]**2 - b_p4[2]**2)
arrs['B_M'] = b_m

In addition to the $B$ mass, we'll also eventually want to look at the masses of possible intermediate particles. It turns out we just need to look at the combination `H1`+`H2`, and `H1`+`H3`. Add some code in the cell below to calculate the four-momenta and masses of these combinations. Then add the masses to `arrs` as new variables `H12_M` and `H13_M`.

In [ ]:
# Calculate H12_M and H13_M here.

Now that we've calculated some masses, let's make a plot. We'll make a histogram of the $B$ mass. To histogram some data, we divide the range of that data into bins and count how many datapoints fall in each bin. Both `matplotlib` and `numpy` have useful functions to help us do this.

In [ ]:
plt.clf()
ax = plt.gca()
# Try playing with the range and number of bins and see how the look of the
# histogram changes. You can also replace bins with an array of bin edges if you
# want to try uneven binning.
ax.hist(arrs['B_M'], range=(5100, 5500), bins=100)
ax.set_xlim(5100, 5500)
# Always label your axes! For histograms, people will usually understand your
# plot even if you don't label the y-axis, but it will need a label if you want
# to publish it!
ax.set_xlabel(r'$m(K^+K^-K^+)$ [MeV]')
ax.set_ylabel('candidates / bin')
plt.show()

This doesn't look great! The $B^+$ has a mass of about $5280~{\rm MeV}$, but
there's only a small lump there. There's also a lump way up around $5420~{\rm
MeV}$. We'll need to use some additional information to try to get rid of some
background. 

### Selecting data
First, let's try using some particle identification information. Most charged
particles produced in $pp$ collisions are pions, so we probably have a lot of
background from randomly combining pions together. We can try to get a better
sample of kaons by requiring our final-state hadrons to have high `ProbK`. To do
this, we'll use a technique called "indexing by boolean." This is easiest to
understand by looking at an example.

In [ ]:
# Print the number of candidates before the selection.
print(len(arrs['B_M']))
# Perform a selection using indexing by boolean. Only the elements of arrs for
# which the condition inside of the brackets is True will be copied to
# selected_arrs.
selected_arrs = arrs[(arrs['H1_ProbK']>0.3) & (arrs['H2_ProbK']>0.3) & (arrs['H3_ProbK']>0.3)]
# Print the number of candidates after the selection.
print(len(selected_arrs['B_M']))

By making a loose cut on `ProbK`, we've eliminated more than $90\%$ of candidates! Try making a histogram of the $B$ mass just for the selected events.

In [ ]:
# Make a histogram of the masses of the candidates passing the ProbK cuts.

You should now be able to see an extremely clear peak! 

### Significance and sideband subtraction
But can you do better? To decide, we can look at a quantity called significance:
$$ 
S=\frac{N_s}{\sqrt{N_s+N_b}}
$$
where $N_s$ is the number of signal and $N_b$
is the number of background in the region of the signal. As a rule of thumb, the
larger the significance of your signal, the greater the statistical precision of
the measurement you're trying to perform. When you're trying to tune cuts for an
event selection, maximizing significance is a good place to start (usually using
a sample of simulated data to avoid accidentally creating a fake peak).

Let's define our signal region as $5200-5360~{\rm MeV}$. Then we can define
"sidebands" on either side of the signal region (SR). If the sidebands are
equidistant from the SR, cover the same total range as the SR, and the
background varies linearly, then the number of candidates in the sidebands will
equal the number of background candidates in the SR. This method of background
estimation is called "sideband subtraction." In practice, the conditions to make
sideband subtraction work are never truly satisifed, but it'll do well enough
for this exercise. We'll define sideband regions as $5120-5200~{\rm MeV}$ and
$5360-5440~{\rm MeV}$.

In [ ]:
# Get the data in the signal region.
selected_arrs_sr = selected_arrs[(selected_arrs['B_M']>5200) & (selected_arrs['B_M']<5360)]
# Get the data in the sideband region.
selected_arrs_sb = selected_arrs[((selected_arrs['B_M']>5120) & (selected_arrs['B_M']<5200)) | ((selected_arrs['B_M']>5360) & (selected_arrs['B_M']<5440))]

# Try making histograms of the signal and sidebands!

# Calculate the significance.
n_bkg = len(selected_arrs_sb['B_M'])
n_sig = len(selected_arrs_sr['B_M']) - n_bkg

significance = n_sig / np.sqrt(n_sig + n_bkg)
print(f'Significance: {significance}')

Now see if you can improve this. Start with the original arrays `arrs` and try coming up with your own event selection. You can start by varying the `ProbK` cut. Then you can try making sure the final-state particles aren't too pion-like by making a `ProbPi` cut. You could also try making sure that the final-state hadrons come from a long-lived hadron decay by cutting on `IPChi2`. Make some cuts, plot the $B$ mass of the selected candidates, and calculate the significance. See if you can beat the number we got above.

#### Note: a word of caution
If you're not careful, it's very easy to bias your measurement with your selection. For example, if you're searching for $CP$ violation, this will vary depending on the decay time of the $B$ meson. If you cut on the $B$ flight distance or final-state hadron `IPChi2`, you can bias the $B$ decay time distribution. But sometimes biased selections are unavoidable. The LHCb software trigger, which selects interesting events to save as the data is collected, makes heavy use of biased selections. In this case you need to study and potentially correct for the resulting bias later in the analysis.

In [ ]:
# Make your own selection, plot the B mass, and calculate the significance of the signal.